# setup

In [5]:
import warnings
warnings.filterwarnings('ignore')

import os
import scipy
from tqdm import tqdm
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import image
from sklearn import linear_model
from brainiak.searchlight.searchlight import Searchlight
from brainiak.searchlight.searchlight import Cube

In [9]:
nb_name = ''

data_dir = '/jukebox/ntb/projects/sketchloop02/data'
SAVE_PATH = '/jukebox/ntb/projects/sketchloop02/data/searchlight_output'
curr_dir = '/jukebox/ntb/projects/sketchloop02/prototype/link'
roi_dir = os.path.join(data_dir, 'copes/roi')
cope_dir = os.path.join(data_dir, 'copes/recog/objectGLM')
sub_dirs = sorted(os.listdir(roi_dir))
sub_dirs = [each[:7] for each in sub_dirs]

## root paths
os.path.abspath(os.path.join(curr_dir,'..','..'))
proj_dir = os.path.abspath(os.path.join(curr_dir,'..','..')) ## use relative paths
feat_dir = os.path.abspath(os.path.join(proj_dir,'data/features')) ## use relative paths 'D:\\data'

# null mask
phases = ['34']
null_mask = np.ones((88, 128, 128))
normalize_on = True
logged = False
iv = 'Unnamed: 0'
affine = np.array([[-1.996683955192566, -0.026332620531320572, -0.11206881701946259, 91.78023529052734],
                   [-0.026291240006685257, 1.9998265504837036, -0.0014756681630387902, -125.46440124511719],
                   [-0.11207851767539978, 7.630718279472148e-09, 1.9968571662902832, -120.91204833984375],
                   [0.0, 0.0, 0.0, 1.0]])

In [10]:
def load_draw_metadata(subject):
    metadata = pd.read_csv(
        '/jukebox/ntb/projects/sketchloop02/data/feature_matrices_and_metadata/metadata_' +
        subject + '_drawing.csv')
    return metadata

# z-score normalization to de-mean & standardize variances within-voxel
def normalize(X):
    X = X - X.mean(0)
    X = X / np.maximum(X.std(0), 1e-5)
    return X

# define and run searchlight
How searchlight works with brainiak:

1. Initiate a searchlight object, articulating certain parameters (e.g., searchlight shape, radius, max edge length, in voxels, of the 3D block(?)).
2. Distribute data to be searched to the searchlight object, sorting between MPI ranks (idk what MPI ranks are tbh).
3. Broadcast data, i.e., define other variables to be available for each execution of the searchlight function.
4. Run the searchlight, this time articulating as a parameter the function to be applied at each searchlight location.

We want to perform searchlight again, but this time the objective is to perform a version of helpers.make_drawing_predictions on each searchlight mask.

In [28]:
def prepostclf(subject_data, mask, sl_rad, bcast_var):
    t1, t2 = trained_objs
    subject_data = subject_data[0].transpose((3, 0, 1, 2))
    
    # format the train/test split
    x_A = subject_data[:80,:].reshape(80, -1)
    x_B = subject_data[80:, :].reshape(80, -1)
    y_A = RM.label.values[:80]
    y_B = RM.label.values[80:]
    
    # normalize if we want
    if normalize_on:
        x_A = normalize(x_A)
        x_B = normalize(x_B)
        
    # dual classifiers
    clfA = linear_model.LogisticRegression(
        penalty='l2',C=1).fit(x_A, y_A)
    clfB = linear_model.LogisticRegression(
        penalty='l2',C=1).fit(x_B, y_B)
    
    ## add prediction probabilities to metadata matrix
    ## must sort so that trained are first, and control is last
    ## also save out new columns in the same order
    _ordering = np.argsort(np.hstack((trained_objs,control_objs))) ## e.g., [chair table bench bed] ==> [3 2 0 1]
    ordering = np.argsort(_ordering) ## get indices that sort from alphabetical to (trained_objs, control_objs)
    probsA = (np.log(clfA.predict_proba(x_B)) if logged else clfA.predict_proba(x_B))
    probsB = (np.log(clfB.predict_proba(x_A)) if logged else clfB.predict_proba(x_A))
    probs = np.concatenate((probsA, probsB), axis=0)
    
    out = probs[:,ordering]
    RM['t1_prob'] = out[:,0]
    RM['t2_prob'] = out[:,1]
    RM['c1_prob'] = out[:,2]
    RM['c2_prob'] = out[:,3]
    RM['t1_max'] = pd.Series(np.logical_and(np.logical_and(out[:,0] > out[:,1], out[:,0] > out[:,2]), out[:,0] > out[:,3]))
    RM['t2_max'] = pd.Series(np.logical_and(np.logical_and(out[:,1] > out[:,0], out[:,1] > out[:,2]), out[:,1] > out[:,3]))
    
    # return proportion of times target > others
    if RM[RM.label==t2].shape[0] + RM[RM.label==t1].shape[0]:
        return (RM[(RM.label==t2) & (RM['t2_max'])].shape[0] + RM[(RM.label==t1) & (RM['t1_max'])].shape[0])/(
            RM[RM.label==t2].shape[0] + RM[RM.label==t1].shape[0])
    else:
        return 0

In [ ]:
# function that sets up and organizes searchlight over a set of subjects
def searchlight_over_each(subjects, sl_rad, phases):
    global RM, DM, trained_objs, control_objs, subject
    
    for subject in tqdm(subjects):
        print(subject)
        
        # set up searchlight object
        sl = Searchlight(sl_rad=sl_rad, shape=Cube)
        
        # arrange data to be distributed to searchlight
        # list of 4D not 3D arrays
        subject_data = []
        for phase in phases:
            this_file = '{}/recog/{}_{}_featurematrix.npy'.format(
                feat_dir, subject, phase)
            subject_data.append(np.load(this_file).transpose((2, 3, 1, 0)))
        subject_data = np.stack(subject_data, axis=0)
        
        # load metadata
        this_file = "{}/recog/metadata_{}_{}.csv".format(feat_dir, subject, phase)
        RM, DM = pd.read_csv(this_file), load_draw_metadata(subject)
        trained_objs = np.unique(DM.label.values)
        control_objs = [i for i in ['bed','bench','chair','table'] if i not in trained_objs]
        
        # distribute and broadcast needed data to searchlight
        sl.distribute(subject_data, null_mask)
        sl.broadcast(None)
        
        # run searchlight
        subject_outputs = np.array(sl.run_searchlight(prepostclf))
        print(np.shape(subject_outputs.astype(np.float32)))
        img = nib.Nifti1Image(subject_outputs.astype(np.float32), affine)
        
        # store output for this subject
        if not os.path.exists(SAVE_PATH):
            os.makedirs(SAVE_PATH)
        
        localizer_tmap_filename = os.path.join(SAVE_PATH, subject + 'prepostacc_{}_searchlight.nii.gz'.format(phase))
        nib.save(img, localizer_tmap_filename)
        
# run searchlight over all subjects and print time to compute results
searchlight_over_each(sub_dirs, 3, phases)

## Conversion of Maps into Standard Space

Using the Jupyter Magic for executing terminal commands, call FSL's flirt function on each searchlight map to convert them into standard space.

In [ ]:
import os
from nilearn import plotting
import matplotlib.pyplot as plt

for sub in sub_dirs:
    print(sub)
    flirtin = os.path.join(SAVE_PATH, sub+'prepostacc_56_searchlight.nii.gz')
    flirtout = os.path.join(SAVE_PATH, sub+'_standard_prepostacc_56_searchlight.nii.gz')
    flirtref = '/jukebox/ntb/projects/sketchloop02/searchlight_outputs/MNI152_T1_2mm_brain.nii.gz'
    flirtinit = '/jukebox/ntb/users/jwammes/sketchloop/subjects/' + sub + '_neurosketch/analysis/firstlevel/draw_run_1.feat/reg/highres2standard.mat'
    !module load fsl; flirt -in "$flirtin" -out "$flirtout" -ref "$flirtref" -applyxfm -init "$flirtinit"